# 多输入多输出通道
在之前的卷积层中，我们经常会有Tensor.reshape(1,1,x,x)这种操作，第一个1是batch，第二个1是1维度，然而图片一般是收纳通道，例如256*256的图片，它的尺寸是3,256,256

当输入包含多个通道的时候，我们要构造一个与输入数据具有相同输入通道数的卷积核，以便于输入数据进行互相关运算。

In [14]:
import torch as t
import torch.nn as nn
from torch import Tensor
conv = nn.Conv2d(3,3,2)

for param in conv.parameters():
    print(param,param.shape)
# 我们可以看到参数的shape = torch.Size([3, 3, 2, 2])
# 意思是3个outchannel 3个 inchannel 2*2的卷积核大小

Parameter containing:
tensor([[[[-0.0810,  0.2035],
          [ 0.0982,  0.1462]],

         [[ 0.1033, -0.2700],
          [-0.2734, -0.1603]],

         [[-0.2174, -0.1042],
          [-0.0885,  0.0224]]],


        [[[-0.1758,  0.1863],
          [-0.0014, -0.1177]],

         [[ 0.2787, -0.0716],
          [ 0.1335, -0.2635]],

         [[ 0.2304,  0.2245],
          [-0.1260, -0.1318]]],


        [[[-0.0032,  0.0122],
          [ 0.2438,  0.1136]],

         [[-0.0258,  0.1951],
          [ 0.1731, -0.1432]],

         [[-0.0817, -0.1634],
          [ 0.0656,  0.0287]]]], requires_grad=True) torch.Size([3, 3, 2, 2])
Parameter containing:
tensor([-0.0071,  0.1748,  0.1445], requires_grad=True) torch.Size([3])


In [15]:
def corr2d(X: Tensor, K: Tensor) -> Tensor:
    """计算二维互相关运算,len(X.shape)=len(K.shape)=2"""
    h, w = K.shape
    Y = t.zeros((X.shape[0]-h+1, X.shape[1]-w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i+h, j:j+w]*K).sum()
    return Y
# 定义多输入通道互相关逆运算，对每个通道执行互相关操作
def corr2d_multi_in(X:Tensor,K:Tensor)->Tensor:
    return sum(corr2d(x,k) for x,k in zip(X,K))

In [16]:
# 下面是一个2通道3*3图片的示例
X = t.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = t.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
print(X.shape,K.shape)
corr2d_multi_in(X, K)


torch.Size([2, 3, 3]) torch.Size([2, 2, 2])


tensor([[ 56.,  72.],
        [104., 120.]])

# 多输出通道
每一层我们都要保证有多个输出通道是至关重要的，在流行的神经网络中，随着网络层数的加深，我们常常会增加输出通道的维度，通过减少空间分辨率以获得更大的通道深度。

由Ci和Co分别代表输入和输出通道的数目，并使用Kh和Kw来表示卷积核的高度和宽度，为了获得多个通道的输出，我们可以为每个输出通道创建一个形状为Ci\*Kh\*Kw的卷积核张量，有Co个通道，所以一共卷积核的形状是Co\*Ci\*Kh\*Kw，*这就是我们刚刚看到的nn.Conv2d中的参数的意义

In [17]:
# 实现多个通道的输入和输出的互相关函数
def corr2d_multi_in_out(X:Tensor,K:Tensor)->Tensor:
    return t.stack([corr2d_multi_in(X,k) for k in K],0)

K = t.stack((K,K+1,K+2),0)
K.shape

torch.Size([3, 2, 2, 2])

In [18]:
corr2d_multi_in_out(X,K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

# 1*1卷积层
1*1卷积不能提取相关的特征，但是它可以提取通道的特征。

In [19]:
def corr2d_multi_in_out_1x1(X:Tensor,K:Tensor)->Tensor:
    c_i,h,w=X.shape
    c_o=K.shape[0]
    X=X.reshape((c_i,h*w))
    K=K.reshape((c_o,c_i))
    Y=t.matmul(K,X)
    return Y.reshape((c_o,h,w))

X = t.normal(0, 1, (3, 3, 3))
K = t.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(t.abs(Y1 - Y2).sum()) < 1e-6